In [ ]:
import tensorflow as tf
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train.shape, y_train.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

X_train = X_train / 255
X_valid = X_valid / 255
X_test = X_test / 255

In [ ]:
X_train.shape, y_train.shape, X_valid.shape, y_train.shape, 

In [ ]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

rough_model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

rough_params = {
    'n_neurons' : list(range(100, 1000)),
    'n_hidden' : [2, 3, 4],
    'learning_rate' : loguniform(10 ** -3, 1)
}

rough_search = RandomizedSearchCV(rough_model, rough_params, n_iter=100, cv=2)
rough_search.fit(X_train, y_train, epochs=10000,
           validation_data=(X_valid, y_valid),
           callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
rough_search.best_params_ # {'learning_rate': 0.2832557832529829, 'n_hidden': 3, 'n_neurons': 653}

In [ ]:
rough_search.best_score_ # 0.9781555533409119

In [ ]:
model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

params = {
    'n_neurons' : [653],
    'n_hidden' : [3],
    'learning_rate' : loguniform(.1, 1)
}

search = RandomizedSearchCV(model, params, n_iter=100, cv=2)
search.fit(X_train, y_train, epochs=10000,
           validation_data=(X_valid, y_valid),
           callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
search.best_params_ # {'learning_rate': 0.39166291079800697, 'n_hidden': 3, 'n_neurons': 653}

In [ ]:
search.best_score_ # 0.9782000184059143